In [1]:
%%time 

###########################################################
# Parte 1
# Configurações gerais de log e warning
#############################################################

#Importação dos módulos desta parte
from datetime import datetime
import logging
import warnings

t0 = datetime.now()
#Criação e configuração do log
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG) 
logFormatter = logging.Formatter('"%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s"')
fh = logging.FileHandler('/home/luis/Downloads/Ex2-3 V{0}.log'.format(datetime.now()))
fh.setLevel(logging.DEBUG)
fh.setFormatter(logFormatter)
log.addHandler(fh)
ch = logging.StreamHandler()
ch.setFormatter(logFormatter)
ch.setLevel(logging.DEBUG)
log.addHandler(ch)

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
warnings.warn = warn
 
#log das versões utilizadas
import sklearn
import scipy

log.info("Versões utilizadas:")
log.info("Sklearn: %s" % sklearn.__version__)
log.info("Scipy: %s" % scipy.__version__)
    
    
log.info("Start!")

###########################################################
# Parte 2
# Importação da base QM7 e separação dos dados de treino e teste
#############################################################

#Importação dos módulos desta parte
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np

t1 = datetime.now()
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Início da parte 2: ")
log.info("Importação da base QM7 e separação dos dados de treino e teste")

#https://www.kaggle.com/mjmurphy28/predicting-atomization-energy-qm7
#The data file (.mat format, we recommend using scipy.io.loadmat for python users) contains five arrays:
#
#"X" - (7165 x 23 x 23), Coulomb matrices
#"T" - (7165), atomization energies (unit: kcal/mol)
#"P" - (5 x 1433), cross-validation splits as used in [Montavon et al. NIPS, 2012]
#"Z" - (7165 x 23), atomic charges
#"R" - (7165 x 23 x 3), cartesian coordinate (unit: Bohr) of each atom in the molecules
mat = loadmat('/home/luis/Dropbox/Pós Graduacao/qm7.mat')
X3D = mat['X'] # Coulomb matrices 3D (7165 x 23 x 23)
y = mat['T'].T #atomization energies


# Separação das partes de X
X_CM = X3D.reshape(7165,529)
#num_atoms = 23
#X_CM = np.zeros((X3D.shape[0], num_atoms*(num_atoms+1)//2), dtype=float)
#iu = np.triu_indices(num_atoms,k=0) 
#iu_dist = np.triu_indices(num_atoms,k=1) # for the pairwise distance matrix, all diagonol entries will be 0 
#X_eigs = np.zeros((X3D.shape[0], num_atoms), dtype=float)
#X_centralities = np.zeros((X3D.shape[0], num_atoms), dtype=float)
#X_interatomic_dist = np.zeros((X3D.shape[0], ((num_atoms*num_atoms)-num_atoms)//2), dtype=float) 
log.info("Feita a leitura do arquivo qm7.mat")

#Separação dos dados de treino e teste
train_X, val_X, train_y, val_y = train_test_split(X_CM, y, random_state=1)
log.info("Feita a separação dos dados de treino e teste")

#maca fim do bloco e tempo gasto
log.info("Tempo gasto na Parte 2: %s" % (datetime.now() - t1))

###########################################################
# Parte 3
# Avaliação dos parametros com o GridSearchCV
#############################################################


#Importação dos módulos desta parte
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd

t1 = datetime.now()
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Início da parte 3: ")
log.info("Avaliação dos parametros com o GridSearchCV")

# Array para guardar os melhores parâmetros de cada execução
best_params = []

# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 5
# gamma: 0.1, 0.3, 0.5, 0.8
# degree: Fibonacci de 1 a 5
# coef0: 0.1, 0.3, 0.5, 0.8
param_KR = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]},  
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8],
                     'degree': [1, 2, 3],
                     'coef0': [0.1, 0.3, 0.5, 0.8]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
param_SVR = [{'kernel': ['linear'],  #o kernel linear só com um parametro demrou 20mn. para executar 
                     'C': [1]}, #, 2, 3, 5]}, 
                    {'kernel': ['rbf'], 
                     'C': [1], # 2, 3, 5], 
                     'gamma': [0.1]}, # 0.3, 0.5, 0.8]},  
                    # {'kernel': ['polynomial'], 
                    # 'C': [1], #2, 3, 5], 
                    # 'gamma': [0.1], # 0.3, 0.5, 0.8],
                    # 'degree': [1], #2, 3, 5],
                    # 'coef0': [0.1]}, # 0.3, 0.5, 0.8]},
                   # {'kernel': ['cosine'], 
                   #  'C': [1, 2, 3, 5]}, 
                   # {'kernel': ['laplacian'], 
                   #  'C': [1, 2, 3, 5], 
                   #  'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
estimators = [{"name": "KernelRidge",
               "estimator": KernelRidge(),
              "parameters": param_KR},
              {"name": "SVR",
              "estimator": SVR(),
             "parameters": param_SVR},
          ]
log.info("Criados os parâmetros para os estimators: {0}".format(estimators))

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV: {0}".format(scores))

#laço para avaliação dos estimators
for estimator in estimators:
    # esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
    #mas foi usada em um laço de "for" para melhor clareza no log de execução
    for score in scores: 

        log.info("-----------------------------------------------------------")
        log.info("# Avaliando: Estimator={0} e Score = {1}".format(estimator["name"], score))

        clf = GridSearchCV(estimator["estimator"], estimator["parameters"], cv=2, verbose=5,  
                   scoring='%s' % score, n_jobs=-1)
        log.info("Criado o objeto GridSearchCV:{0}".format(clf))

        clf.fit(train_X, train_y)
        log.info("Concluído o 'fit' com os dados de treino.")

        log.info("Melhore estimator encontrado: {0}".format(clf.best_estimator_))
        log.info("Melhores parâmetros encontrados: {0}".format(clf.best_params_))
        log.info("Melhore score encontrado: {0:9.3f}".format(clf.best_score_))
        
        #Guarda os melhores parametros desta execução
        best_params.append("score={sc} - {bs:9.3f} -> {be}".format(sc = score, 
                                                                      bs=clf.best_score_, 
                                                                      be=clf.best_estimator_))
        # [clf.best_params_, clf.best_score_, clf.best_estimator_]
        
        
        log.info("Resultados dos testes com os diversos parâmetros: ")
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            log.info("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))

# exibe no log os melhores parâmetros de todas as execuções
log.info("")
log.info("-----------------------------------------------------------")
log.info("Melhores parâmetros de todas as execuções: ")
for be in best_params:
    log.info(be)
log.info("")

# maca fim do bloco e tempo gasto
log.info("Tempo gasto na Parte 3: %s" % (datetime.now() - t1))


####################################################################
# Fim do estudo
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Finish!")


"2019-05-06 17:46:02,748 [MainThread  ] [INFO ]  Versões utilizadas:"
"2019-05-06 17:46:02,750 [MainThread  ] [INFO ]  Sklearn: 0.19.1"
"2019-05-06 17:46:02,751 [MainThread  ] [INFO ]  Scipy: 1.2.1"
"2019-05-06 17:46:02,752 [MainThread  ] [INFO ]  Start!"
"2019-05-06 17:46:02,964 [MainThread  ] [INFO ]  "
"2019-05-06 17:46:02,965 [MainThread  ] [INFO ]  ------------------------------------------------------"
"2019-05-06 17:46:02,966 [MainThread  ] [INFO ]  "
"2019-05-06 17:46:02,966 [MainThread  ] [INFO ]  Início da parte 2: "
"2019-05-06 17:46:02,967 [MainThread  ] [INFO ]  Importação da base QM7 e separação dos dados de treino e teste"
"2019-05-06 17:46:03,003 [MainThread  ] [INFO ]  Feita a leitura do arquivo qm7.mat"
"2019-05-06 17:46:03,013 [MainThread  ] [INFO ]  Feita a separação dos dados de treino e teste"
"2019-05-06 17:46:03,014 [MainThread  ] [INFO ]  Tempo gasto na Parte 2: 0:00:00.050187"
"2019-05-06 17:46:03,300 [MainThread  ] [INFO ]  "
"2019-05-06 17:46:03,300 [MainThr

Fitting 2 folds for each of 232 candidates, totalling 464 fits
[CV] alpha=1, kernel=linear ..........................................
[CV] alpha=1, kernel=linear ..........................................
[CV] alpha=2, kernel=linear ..........................................
[CV] alpha=2, kernel=linear ..........................................
[CV] . alpha=1, kernel=linear, score=0.8734938169263304, total=  20.3s
[CV] alpha=3, kernel=linear ..........................................
[CV] . alpha=2, kernel=linear, score=0.9031273863605731, total=  22.3s
[CV] alpha=3, kernel=linear ..........................................
[CV] . alpha=3, kernel=linear, score=0.9429316145038812, total=   2.1s
[CV] alpha=5, kernel=linear ..........................................
[CV] . alpha=1, kernel=linear, score=0.8974310036608082, total=  23.4s
[CV] alpha=5, kernel=linear ..........................................
[CV] . alpha=3, kernel=linear, score=0.9465385470383434, total=   1.8s
[CV] alpha=1, 

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   29.0s


[CV]  alpha=1, gamma=0.3, kernel=rbf, score=-47.19132358877318, total=   2.3s
[CV] alpha=1, gamma=0.8, kernel=rbf ..................................
[CV]  alpha=1, gamma=0.1, kernel=rbf, score=-47.178916339921656, total=   3.2s
[CV] alpha=1, gamma=0.8, kernel=rbf ..................................
[CV]  alpha=1, gamma=0.5, kernel=rbf, score=-46.214764483190166, total=   2.5s
[CV] alpha=2, gamma=0.1, kernel=rbf ..................................
[CV]  alpha=1, gamma=0.5, kernel=rbf, score=-47.191350837729615, total=   2.4s
[CV] alpha=2, gamma=0.1, kernel=rbf ..................................
[CV]  alpha=1, gamma=0.8, kernel=rbf, score=-46.214764579728914, total=   2.4s
[CV] alpha=2, gamma=0.3, kernel=rbf ..................................
[CV]  alpha=1, gamma=0.8, kernel=rbf, score=-47.191350936484724, total=   2.3s
[CV] alpha=2, gamma=0.3, kernel=rbf ..................................
[CV]  alpha=2, gamma=0.1, kernel=rbf, score=-47.18302946336176, total=   2.5s
[CV] alpha=2, gamma=0.5

[CV]  alpha=1, coef0=0.1, degree=3, gamma=0.5, kernel=polynomial, score=0.7943987676151365, total=  27.6s
[CV] alpha=1, coef0=0.3, degree=1, gamma=0.1, kernel=polynomial ......
[CV]  alpha=1, coef0=0.1, degree=3, gamma=0.8, kernel=polynomial, score=0.6890284295384509, total=  25.9s
[CV] alpha=1, coef0=0.3, degree=1, gamma=0.3, kernel=polynomial ......
[CV]  alpha=1, coef0=0.1, degree=3, gamma=0.8, kernel=polynomial, score=0.7959092631085375, total=  26.7s
[CV] alpha=1, coef0=0.3, degree=1, gamma=0.3, kernel=polynomial ......


...

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, score=-49985.60597522948, total=  13.1s
[CV]  C=1, gamma=0.1, kernel=rbf, score=-51090.92183206494, total=  13.4s


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   20.0s remaining:   20.0s


[CV] .... C=1, kernel=linear, score=-1423.3106819647974, total=  25.6s
[CV] .... C=1, kernel=linear, score=-1357.5926841347616, total=  27.5s


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   32.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   32.3s finished
"2019-05-06 19:29:03,627 [MainThread  ] [INFO ]  Concluído o 'fit' com os dados de treino."
"2019-05-06 19:29:03,628 [MainThread  ] [INFO ]  Melhore estimator encontrado: SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)"
"2019-05-06 19:29:03,629 [MainThread  ] [INFO ]  Melhores parâmetros encontrados: {'C': 1, 'kernel': 'linear'}"
"2019-05-06 19:29:03,630 [MainThread  ] [INFO ]  Melhore score encontrado: -1390.446"
"2019-05-06 19:29:03,631 [MainThread  ] [INFO ]  Resultados dos testes com os diversos parâmetros: "
"2019-05-06 19:29:03,632 [MainThread  ] [INFO ]  -1390.446 (+/-65.718) for {'C': 1, 'kernel': 'linear'}"
"2019-05-06 19:29:03,633 [MainThread  ] [INFO ]  -50538.367 (+/-1105.316) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}"
"2019-

CPU times: user 9min 9s, sys: 18.5 s, total: 9min 27s
Wall time: 1h 43min 1s
